In [3]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:

#import new data, use time to see how long it takes to download data
%time df=pd.read_csv('../Marvel_Fun/fivethirtyeight-comic-characters-dataset/marvel-wikia-data.csv')
df.head()

CPU times: user 49.5 ms, sys: 10.9 ms, total: 60.4 ms
Wall time: 99.3 ms


,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year
0,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962.0
1,7139,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941.0
2,64786,"Wolverine (James \""Logan\"" Howlett)",\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974.0
3,1868,"Iron Man (Anthony \""Tony\"" Stark)",\/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963.0
4,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950.0


In [5]:
#change the naming convention of the features
df.columns=[i.replace(' ', '_').lower() for i in df.columns]

In [50]:
#use a function to visualize the data

def data_brief(df):
    d_type = df.dtypes
    columns = pd.DataFrame(df.columns, d_type, columns = ['Features:']) #columns
    
    print(f'\nDataTypes:', columns)
    print(f'\nNum of Features:', len(df.columns))
    print(f'\nNum of Rows:', df.shape)
    print(f'\nIndex:', df.index.name)
    
    return

data_brief(df)    


DataTypes:                 Features:
int64             page_id
object               name
object            urlslug
object                 id
object              align
object                eye
object               hair
object                sex
object                gsm
object              alive
float64       appearances
object   first_appearance
float64              year

Num of Features: 13

Num of Rows: (16376, 13)

Index: None


Most of the data is categorical.  I'll use this information when making the statistical function to further inspect the data.

### Inspect basic statistics of the data

I prefer to create a function to analyze the data.

In [15]:
def df_analysis(df):
    eda_df={} #create an open df
    eda_df['count'] = df.count()
    eda_df['null ct'] = df.isnull().sum()
    eda_df['null %'] = df.isnull().mean()
    eda_df['min'] = df.min()
    eda_df['max'] = df.max()
    eda_df['mean'] = df.mean()
    eda_df['median'] = df.median()
    
    return pd.DataFrame(eda_df)

df_analysis(df)

,count,null ct,null %,dtype,min,max,mean,median
align,13564,2812,0.171715,object,NaN,NaN,NaN,NaN
alive,16373,3,0.000183,object,NaN,NaN,NaN,NaN
appearances,15280,1096,0.066927,float64,1,4043,17.033377,3.0
eye,6609,9767,0.596422,object,NaN,NaN,NaN,NaN
first_appearance,15561,815,0.049768,object,NaN,NaN,NaN,NaN
gsm,90,16286,0.994504,object,NaN,NaN,NaN,NaN
hair,12112,4264,0.260381,object,NaN,NaN,NaN,NaN
id,12606,3770,0.230215,object,NaN,NaN,NaN,NaN
name,16376,0,0.000000,object,'Spinner (Earth-616),\u00c4kr\u00e4s (Earth-616),NaN,NaN
page_id,16376,0,0.000000,int64,1025,755278,300232.082377,282578.0


From this analysis, I see that most of the data is categorical, which makes total sense.  Due to this, most 

In [10]:
#going to remove data (columns) I think is unnecessary to the matters at hand
#keep sex but much of the data is null

#data.dropna(subset=['align'], inplace=True)

#data.ALIGN.replace(['neutral_characters', 'good_characters','bad_characters'], [0,0,1], inplace=True)



In [11]:
#create a function to take a dataframe as input and calculate summary statistics to reveal insights about the data



,null_sum,null_pct,dtypes,count,mean,median,min,max
align,2812,0.171715,object,13564,NaN,NaN,NaN,NaN
alive,3,0.000183,object,16373,NaN,NaN,NaN,NaN
appearances,1096,0.066927,float64,15280,17.033377,3.0,1,4043
eye,9767,0.596422,object,6609,NaN,NaN,NaN,NaN
first_appearance,815,0.049768,object,15561,NaN,NaN,NaN,NaN
gsm,16286,0.994504,object,90,NaN,NaN,NaN,NaN
hair,4264,0.260381,object,12112,NaN,NaN,NaN,NaN
id,3770,0.230215,object,12606,NaN,NaN,NaN,NaN
name,0,0.000000,object,16376,NaN,NaN,'Spinner (Earth-616),\u00c4kr\u00e4s (Earth-616)
page_id,0,0.000000,int64,16376,300232.082377,282578.0,1025,755278


In [12]:
#how much is null without the entire df?
#quick and dirty
data.isnull().sum()

page_id                 0
name                    0
urlslug                 0
id                   3770
align                2812
eye                  9767
hair                 4264
sex                   854
gsm                 16286
alive                   3
appearances          1096
first_appearance      815
year                  815
dtype: int64

In [13]:
#create a data frame to explore null values.  I like this better
null_df=pd.DataFrame(data.isnull().sum(), columns=['Count of Nulls'])
null_df.index.name="Column"
null_df.sort_values(['Count of Nulls'], ascending=False).head(10)

,Count of Nulls
Column,
gsm,16286
eye,9767
hair,4264
id,3770
align,2812
appearances,1096
sex,854
first_appearance,815
year,815


#check out datatypes
data.dtypes.value_counts()

In [14]:
#check out the object datatypes 
data.select_dtypes(include=['object']).columns

Index(['name', 'urlslug', 'id', 'align', 'eye', 'hair', 'sex', 'gsm', 'alive',
       'first_appearance'],
      dtype='object', name='Column')

I can see that some of the columns are categorical and can be converted into numeric valaues
1. align 
2. sex 
3. alive

In [22]:
#encoding
#replace neutral and good characters to 0, bad characters to 1
#data[(data['Neutral Characters','Good Characters'] == 0) & data['Bad Characters']==1].head(3)

obj_df = data.select_dtypes(include=['object']).copy()
obj_df.head(3)

Column,name,urlslug,id,align,eye,hair,sex,gsm,alive,first_appearance
0,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,Aug-62
1,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,Mar-41
2,"Wolverine (James \""Logan\"" Howlett)",\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,Oct-74


In [15]:
#i have a hard time remembering column names so I am going to put it in a variable
data_col= data.columns
data_col

Index(['page_id', 'name', 'urlslug', 'id', 'align', 'eye', 'hair', 'sex',
       'gsm', 'alive', 'appearances', 'first_appearance', 'year'],
      dtype='object', name='Column')

In [16]:
#going to remove data I think is unnecessary to the matters at hand
#keep sex but much of the data is null
#or can use .dropnull()
ad_data=data[['name','id','align','eye','hair','alive']]
ad_data.head(3)

Column,name,id,align,eye,hair,alive
0,Spider-Man (Peter Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Living Characters
1,Captain America (Steven Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Living Characters
2,"Wolverine (James \""Logan\"" Howlett)",Public Identity,Neutral Characters,Blue Eyes,Black Hair,Living Characters


Going to remove all columns containing NULL in the 'align' column and convert the columns to numbers.

1.  evil
2.  good/neutral

In [23]:
#group align column by aliance
align_work=data.groupby('align')

align_work.first()

Column,page_id,name,urlslug,id,eye,hair,sex,gsm,alive,appearances,first_appearance,year
align,,,,,,,,,,,,
Bad Characters,1448,Victor von Doom (Earth-616),\/Victor_von_Doom_(Earth-616),Public Identity,Brown Eyes,Brown Hair,Male Characters,Bisexual Characters,Living Characters,721.0,Jul-62,1962.0
Good Characters,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Hazel Eyes,Brown Hair,Male Characters,Bisexual Characters,Living Characters,4043.0,Aug-62,1962.0
Neutral Characters,64786,"Wolverine (James \""Logan\"" Howlett)",\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Blue Eyes,Black Hair,Male Characters,Bisexual Characters,Living Characters,3061.0,Oct-74,1974.0
